### Определите, сайты каких тематик популярны?

По ссылке https://www.liveinternet.ru/rating/ можно посмотреть рейтинги сайтов рунета. На каждой странице представлены 30 сайтов, где рядом с описанием сайта есть количество посетителей за день. 

Для ответа на вопрос предлагается следующий алгоритм:

**1 (2 балла)** Скачаем информацию об описании сайта и количестве посетителей. И так для каждой из первых 20 страниц. Получится топ. 600 сайтов. Считаем, что остальные не смогут внести вклад в определение главых тематик рунета. 

**2 (2 балла)** Разобьем описание сайта на слова, очистим их от знаков препинания и приведем к нижнему регистру. Опционально вы можете лемматизировать слова с помощью библиотеки pymorphy2 (ее сперва придется установить).

**3 (1 балл)** Заведем словарь (или defaultdict), куда в качестве ключей будем добавлять полученные слова, а в качестве значений - суммарное количество посетителей сайтов в описании которых употребляется данное слово.

**4 (2 балла)** Выведем топ 100 популярных слов. Они нам и подскажут самые популярные тематики в рунете.

**5 (1 балл)** Получившийся топ кажется немного странным. Расскажите, как еще можно было бы ответить на вопрос с помощью сайта liveinternet.ru/rating/ ?


### Полезные указания:

**1** если мы попробуем скачать https://www.liveinternet.ru/rating/ с помощью requests, то обнаружим, что в теле ответа – пустышка. Дело в том, что сам рейтинг докачивается позже. Это можно определить, запустив иструменты разработчика в вашем браузере (см. скриншот: https://monosnap.com/file/EwpyHHt0uedwaEHP7kX1yEO3I2i58e и https://monosnap.com/file/II0HQ6aJZGyjmwfWnioTy9fcBoY6O7). Файл с рейтингами (today.tsv) подкачивается по ссылке https://www.liveinternet.ru/rating///today.tsv?page=1 . Параметры этой ссылки page=1 можно менять.

**2** Файл с рейтингами имеет tsv-формат. Это значит, что внутри колонки, разделенные табами. Первая строчка в файле нам не интересна, в конце тоже есть 2 лишних перевода строки. Из остального содержимого легко добывается информация о сайте и количество посетителей за сегодня.


In [26]:
import requests
import re
import pymorphy2

reg = re.compile('[^a-zA-Zа-яА-Я ]')
morph = pymorphy2.MorphAnalyzer()
dict_of_words = {}

for page in range(21)[1::]: 
    resp = requests.get('https://www.liveinternet.ru/rating///today.tsv?page=' + str(page))
    for index, string in enumerate(resp.text.split('\n')):
        sections = string.split('\t')
        if len(sections) < 6:
            continue
        words = reg.sub('', sections[2]).replace('quot','').lower().split()
        for word in words:
            if morph.parse(word)[0].normal_form in dict_of_words.keys():
                dict_of_words[morph.parse(word)[0].normal_form] += int(sections[3])
            else:
                dict_of_words[morph.parse(word)[0].normal_form] = int(sections[3])

dict_of_words_sorted = sorted(dict_of_words.items(), key=lambda x : x[1], reverse = True)

for num in range(101)[1::]: 
    print('{}: {} - {}'.format(str(num), dict_of_words_sorted[num-1][0]
                               , dict_of_words_sorted[num-1][1]))
    
# 5 - можно пройти по ссылке каждого из сайтов рейтинга выкачать их контент 
#     главной страницы и составить словарь популярных слов на основе данных с главных страниц
#     или пойти еще глубже и вы качать весь контент с первых 10 вложенных страниц сайтов рейтинга
#     как это делают поисковые роботы 

1: вконтактёр - 67331592
2: новость - 25188217
3: и - 23966959
4: mailru - 17503390
5: почта - 9770267
6: в - 9238104
7: hearst - 9083441
8: shkulev - 9083441
9: сайт - 8574869
10: главное - 7436857
11: россия - 6636791
12: мир - 6332515
13: digital - 6297229
14: портал - 6162924
15: онлайн - 5946421
16: о - 5663166
17: весь - 5018492
18: украина - 4718515
19: сеть - 4633737
20: информационный - 3917534
21: звезда - 3866937
22: видео - 3778241
23: network - 3757975
24: ответ - 3755618
25: womens - 3703134
26: риа - 3589622
27: час - 3391429
28: комсомольский - 3266561
29: телеканал - 3246405
30: русский - 3227685
31: для - 3077773
32: хороший - 3017788
33: канал - 2986238
34: media - 2982361
35: отзыв - 2841756
36: тв - 2743987
37: форум - 2703433
38: новостной - 2647144
39: от - 2646400
40: поиск - 2638947
41: интернет - 2627754
42: правдаdigital - 2619024
43: сообщество - 2537534
44: на - 2474247
45: rt - 2417341
46: рецепт - 2416427
47: погода - 2356739
48: известие - 2307069
49: ав